In [1]:
%cd /kaggle/working/

/kaggle/working


In [2]:
!git clone https://github.com/VarunGumma/IndicTransTokenizer

Cloning into 'IndicTransTokenizer'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 123 (delta 52), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (123/123), 3.85 MiB | 14.89 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [3]:
%cd /kaggle/working/IndicTransTokenizer

/kaggle/working/IndicTransTokenizer


In [4]:
pip install --editable ./

Obtaining file:///kaggle/working/IndicTransTokenizer
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/VarunGumma/indic_nlp_library to /tmp/pip-install-p1hszo0x/indic-nlp-library-it2_961a96a08f094f70b557c3a8e45b58f9
  Running command git clone --filter=blob:none --quiet https://github.com/VarunGumma/indic_nlp_library /tmp/pip-install-p1hszo0x/indic-nlp-library-it2_961a96a08f094f70b557c3a8e45b58f9
  Resolved https://github.com/VarunGumma/indic_nlp_library to commit 09d30a15286cc252a12682e5450c807379717eaf
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 899.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 7.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.5 MB/s eta 0:00:0000:0

In [7]:
import random
en=[]
hi=[]
gu=[]
# Load English lines
with open("/kaggle/input/datase/sentences_en.txt", "r", encoding="utf-8") as en_file:
    en = en_file.readlines()

# Load Hindi lines
with open("/kaggle/input/datase/sentences_hi.txt", "r", encoding="utf-8") as hi_file:
    hi = hi_file.readlines()
    
# Load Gujrati lines
with open("/kaggle/input/datase/sentences_guj.txt", "r", encoding="utf-8") as guj_file:
    gu = guj_file.readlines()


In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

# Load en-indic

In [10]:
tokenizer = IndicTransTokenizer(direction="en-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-en-indic-dist-200M", trust_remote_code=True,low_cpu_mem_usage=True)
model=model.to("cuda")

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

# English to Hindi

In [11]:
# translation = []
# for sentences in en:
#     batch = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="hin_Deva")
#     batch = tokenizer(batch, src=True, return_tensors="pt")

#     with torch.inference_mode():
#         outputs = model.generate(**batch, num_beams=5, num_return_sequences=1, max_length=256)

#     outputs = tokenizer.batch_decode(outputs, src=False)
#     outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
#     translation.append(outputs)
    
    
translation = []
for i in range(0,len(en),4):
    batch = en[i : i + 4]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="eng_Latn", tgt_lang="hin_Deva")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    translation.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(translation))

250


In [18]:
with open("en_to_hi2.txt", "w", encoding="utf-8") as en_out_file:
    for list in translation:
        for sent in list:
                en_out_file.write(sent.strip() + "\n")

# Load indic-en

In [20]:
tokenizer = IndicTransTokenizer(direction="indic-en")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-en-dist-200M", trust_remote_code=True,low_cpu_mem_usage=True)
model=model.to("cuda")

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/914M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

# Hindi to English

In [21]:
translation = []
for i in range(0,len(hi),4):
    batch = hi[i : i + 4]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="hin_Deva", tgt_lang="eng_Latn")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="eng_Latn")
    translation.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(translation))
print(translation[-1])

250
['The Prime Minister began the virtual consecration of the statue by pressing the button.', 'The law and order situation is being closely monitored.', 'The two countries share vibrant and growing people-to-people ties.', 'This will facilitate students interested in cinema from the northeastern states.']


In [22]:
with open("hi_to_en2.txt", "w", encoding="utf-8") as en_out_file:
    for list in translation:
        for sent in list:
                en_out_file.write(sent.strip() + "\n")

# Load indic-indic

In [23]:
tokenizer = IndicTransTokenizer(direction="indic-indic")
ip = IndicProcessor(inference=True)
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/indictrans2-indic-indic-dist-320M", trust_remote_code=True)
model=model.to("cuda")

config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-dist-320M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# Gujarati to Hindi

In [24]:
translation = []
for i in range(0,len(gu),4):
    batch = gu[i : i + 4]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="guj_Gujr", tgt_lang="hin_Deva")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="hin_Deva")
    translation.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(translation))
print(translation[-1])

250
['प्रधानमंत्री ने बटन दबाकर प्रतिमा के आभासी अभिषेक की शुरुआत की।', 'कानून और व्यवस्था की स्थिति पर कड़ी नजर रखी जा रही है।', 'भारत और पुर्तगाल दोनों के लोगों के बीच जीवंत संबंध हैं और यह जुड़ाव बढ़ रहा है।', 'यह पूर्वोत्तर राज्य के सिनेमा में रुचि रखने वाले छात्रों को सुविधा प्रदान करेगा।']


In [26]:
with open("gu_to_hi2.txt", "w", encoding="utf-8") as en_out_file:
    for list in translation:
        for sent in list:
                en_out_file.write(sent.strip() + "\n")

# Hindi to Gujarati

In [27]:
translation = []
for i in range(0,len(hi),4):
    batch = hi[i : i + 4]

    # Preprocess the batch and extract entity mappings
    batch = ip.preprocess_batch(batch, src_lang="hin_Deva", tgt_lang="guj_Gujr")

    # Tokenize the batch and generate input encodings
    batch = tokenizer(batch, src=True, return_tensors="pt").to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**batch,use_cache=True, num_beams=5, num_return_sequences=1, max_length=256)

    # Decode the generated tokens into text
    outputs = tokenizer.batch_decode(outputs, src=False)

    # Postprocess the translations, including entity replacement
    outputs = ip.postprocess_batch(outputs, lang="guj_Gujr")
    translation.append(outputs)
    del batch
    torch.cuda.empty_cache()
print(len(translation))
print(translation[-1])

250
['પ્રધાનમંત્રીએ બટન દબાવીને પ્રતિમાના વર્ચ્યુઅલ અભિષેકની શરૂઆત કરી હતી.', 'કાયદો અને વ્યવસ્થાની સ્થિતિ પર નજીકથી નજર રાખવામાં આવી રહી છે.', 'બંને દેશો લોકો વચ્ચે જીવંત અને વધતા જતા સંબંધોને વહેંચે છે.', 'આ પૂર્વોત્તર રાજ્યોના ફિલ્મ રસ ધરાવતા વિદ્યાર્થીઓને સુવિધા આપશે.']


In [28]:
with open("hi_to_gu2.txt", "w", encoding="utf-8") as en_out_file:
    for list in translation:
        for sent in list:
                en_out_file.write(sent.strip() + "\n")